In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation

In [ ]:
# # Load the dataset
# file_path = "./dataset_final/patient1/test5.csv"  # Replace with your file path
# df = pd.read_csv(file_path)

# # Define segment lengths (assumed average human proportions in meters)
# thigh_length = 0.4  # Length from hip to knee
# shank_length = 0.4  # Length from knee to ankle

# # Convert gyroscope data to radians (assuming small angle approximation for integration)
# deg_to_rad = np.pi / 180.0

# # Extract relevant gyroscope data for thigh and shank (Right and Left)
# gyro_rt = df["RT_GyroY"].values * deg_to_rad  # Right Thigh (rotation around Y-axis)
# gyro_rs = df["RS_GyroY"].values * deg_to_rad  # Right Shank
# gyro_lt = df["LT_GyroY"].values * deg_to_rad  # Left Thigh
# gyro_ls = df["LS_GyroY"].values * deg_to_rad  # Left Shank

# # Integrate gyroscope data to get estimated angles (assuming constant time step)
# theta_rt = np.cumsum(gyro_rt)  # Right thigh angle
# theta_rs = np.cumsum(gyro_rs)  # Right shank angle
# theta_lt = np.cumsum(gyro_lt)  # Left thigh angle
# theta_ls = np.cumsum(gyro_ls)  # Left shank angle

# # Define function to update stick figure in animation
# def update(frame):
#     ax.clear()
#     ax.set_xlim(-1, 1)
#     ax.set_ylim(-1, 1)
#     ax.set_title("2D Gait Cycle Animation")
#     ax.set_xlabel("X position (m)")
#     ax.set_ylabel("Y position (m)")

#     # Compute joint positions for right leg
#     hip = np.array([0.2, 0.8])  # Fixed reference point (right hip)
#     knee_r = hip + thigh_length * np.array([np.sin(theta_rt[frame]), -np.cos(theta_rt[frame])])
#     ankle_r = knee_r + shank_length * np.array([np.sin(theta_rs[frame] + theta_rt[frame]), -np.cos(theta_rs[frame] + theta_rt[frame])])

#     # Compute joint positions for left leg
#     hip_l = np.array([-0.2, 0.8])  # Fixed reference point (left hip)
#     knee_l = hip_l + thigh_length * np.array([np.sin(theta_lt[frame]), -np.cos(theta_lt[frame])])
#     ankle_l = knee_l + shank_length * np.array([np.sin(theta_ls[frame] + theta_lt[frame]), -np.cos(theta_ls[frame] + theta_lt[frame])])

#     # Plot right leg
#     ax.plot([hip[0], knee_r[0]], [hip[1], knee_r[1]], 'ro-', label="Right Thigh")
#     ax.plot([knee_r[0], ankle_r[0]], [knee_r[1], ankle_r[1]], 'bo-', label="Right Shank")

#     # Plot left leg
#     ax.plot([hip_l[0], knee_l[0]], [hip_l[1], knee_l[1]], 'ro-')
#     ax.plot([knee_l[0], ankle_l[0]], [knee_l[1], ankle_l[1]], 'bo-')

#     ax.legend(loc="upper right")

# # Create figure and animation
# fig, ax = plt.subplots(figsize=(5, 5))
# ani = animation.FuncAnimation(fig, update, frames=len(df), interval=50)

# # Save animation as GIF
# ani.save("gait_animation.gif", writer="pillow", fps=30)
# print('here')
# # Show the animation
# plt.show()


In [1]:
import pandas as pd
import numpy as np
from gaitmap.utils.coordinate_transforms import convert_to_standard

# Load your dataset
file_path = "./dataset_final/patient1/test5.csv"
df = pd.read_csv(file_path)

# Select IMU data (Gyroscope & Accelerometer) for right ankle as an example
imu_data = {
    "right_ankle": {
        "acc": df[["RA_AccelX", "RA_AccelY", "RA_AccelZ"]].values,
        "gyro": df[["RA_GyroX", "RA_GyroY", "RA_GyroZ"]].values,
    },
    "left_ankle": {
        "acc": df[["LA_AccelX", "LA_AccelY", "LA_AccelZ"]].values,
        "gyro": df[["LA_GyroX", "LA_GyroY", "LA_GyroZ"]].values,
    }
}

# Convert to standard IMU format for GaitMap processing
imu_data = convert_to_standard(imu_data)


ModuleNotFoundError: No module named 'gaitmap.utils.coordinate_transforms'

In [ ]:
from gaitmap.stride_segmentation import BarthDtw

# Initialize the stride segmentation model
stride_segmenter = BarthDtw(max_cost=2.5)

# Apply segmentation to the right ankle IMU data
stride_segmenter.segment(imu_data["right_ankle"]["gyro"], sampling_rate_hz=100)

# Extract the detected stride events
segmented_strides = stride_segmenter.stride_list_
print(segmented_strides)


In [ ]:
from gaitmap.gait_features import SpatialTemporalGaitParameters

# Initialize gait parameter computation
gait_params = SpatialTemporalGaitParameters()

# Compute gait parameters based on segmented strides
results = gait_params.compute(imu_data["right_ankle"], segmented_strides)

# Print key gait metrics
print("Step Length:", results["step_length"])
print("Stride Time:", results["stride_time"])
print("Walking Speed:", results["walking_speed"])


In [ ]:
import matplotlib.pyplot as plt

# Plot raw gyroscope data for one stride
stride_idx = segmented_strides[0]  # First detected stride
plt.plot(df.index[stride_idx[0]:stride_idx[1]], df["RA_GyroY"][stride_idx[0]:stride_idx[1]])
plt.title("Gyroscope Y-axis Data During Stride")
plt.xlabel("Time (samples)")
plt.ylabel("Angular Velocity (rad/s)")
plt.show()


In [ ]:
from gaitmap.trajectory_estimation import FootTrajectoryEstimation

# Initialize foot trajectory estimation
trajectory_estimator = FootTrajectoryEstimation()

# Compute foot trajectory based on IMU data
foot_trajectory = trajectory_estimator.estimate_trajectory(imu_data["right_ankle"])

# Plot foot trajectory in 3D
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(foot_trajectory[:, 0], foot_trajectory[:, 1], foot_trajectory[:, 2])
ax.set_title("3D Foot Trajectory")
ax.set_xlabel("X-axis (m)")
ax.set_ylabel("Y-axis (m)")
ax.set_zlabel("Z-axis (m)")
plt.show()
